In [1]:
import os
from torch.utils.data import DataLoader
from src.pipelines.spacat_pipeline import TryOnPipeline
from src.dataset.vitonhd import VITONHDDataset
from src.utils import get_project_root, show

/home/hosjiu/workspace/heatmob/vto-engine/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
PROJECT_ROOT_PATH = get_project_root()

pipeline_path = os.path.join(PROJECT_ROOT_PATH, 'PLACE YOUR SAVED PIPELINE PATH HERE.')
spacat_pipe = TryOnPipeline.from_pretrained(pipeline_path)

ValueError: The provided pretrained_model_name_or_path "/home/hosjiu/workspace/heatmob/vto-engine/PLACE YOUR SAVED PIPELINE PATH HERE." is neither a valid local path nor a valid repo id. Please check the parameter.

In [ ]:
test_dataset = VITONHDDataset(
    data_rootpath=os.path.join(PROJECT_ROOT_PATH, 'datasets', 'vitonhd'),
    use_trainset=False,
    height=512,
    width=384,
    use_CLIPVision=False
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

sing_batch_test = next(iter(test_dataloader))
show(sing_batch_test['original_image'])

SyntaxError: invalid syntax (3750950305.py, line 18)